In [8]:
# Data loading and merging
import os
from glob import glob
import pandas as pd

# Define the path to the datasets
data_path = os.path.join("..","data","raw")

# Get list of Excel files natching *_cars.xlsx
file_paths = glob(os.path.join(data_path, "*_cars.xlsx"))

# Load and merge all Excel files
all_dataframes = []
for file_path in file_paths:
    city_name = os.path.basename(file_path).split('_')[0].capitalize() # Extract city from the filename
    df = pd.read_excel(file_path)
    df['City'] = city_name
    all_dataframes.append(df)

# Concatenate all dataframes
merged_df = pd.concat(all_dataframes, ignore_index=True)

print(f" Merged Data Dataset: \n{merged_df.columns.tolist()}")
print(f" Merged Data Dataset: \n{merged_df.head()}")
print(f" Merged Data Dataset: \n{merged_df.shape}")

# Define the output path
output_path = os.path.join("..","notebooks", "merged_car_data.csv")
merged_df.to_csv(output_path, index=False)


 Merged Data Dataset: 
['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs', 'car_links', 'City']
 Merged Data Dataset: 
                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': '

| Column             | Description                                                               | EDA Usefulness | Action Needed                            |
| ------------------ | ------------------------------------------------------------------------- | -------------- | ---------------------------------------- |
| new_car_detail   | JSON-like string with key info: price, km, fuel, transmission, model year | ✅ High         | ✅ Flatten (already done earlier)         |
| new_car_overview | JSON-like structure with top-level info like registration, insurance      | ⏳ Medium       | Can extract later if needed              |
| new_car_feature  | Feature list (comfort, safety, etc.)                                      | ⏳ Low–Medium   | Optional; not structured for numeric EDA |
| new_car_specs    | Contains mileage, engine size, power, etc. (nested dicts)                 | ✅ High         | 🚧 Needs flattening                      |
| car_links        | URL of listing                                                            | ❌ None         | 🔻 Drop or ignore                        |
| City             | City name                                                                 | ✅ High         | ➕ Use in grouping, distribution          |


| Feature                      | From Column        | Purpose                         |
| ---------------------------- | ------------------ | ------------------------------- |
| `price`                      | new\_car\_detail   | Target variable                 |
| `modelYear`                  | new\_car\_detail   | Car age                         |
| `km`                         | new\_car\_detail   | Mileage usage                   |
| `ft` (fuel)                  | new\_car\_detail   | Categorical grouping            |
| `transmission`               | new\_car\_detail   | Categorical grouping            |
| `ownerNo`                    | new\_car\_detail   | Ownership status                |
| `City`                       | (already separate) | Regional analysis               |
| `engine`, `power`, `mileage` | new\_car\_specs    | Feature importance, correlation |


In [25]:
# Define the merged_data_path
merged_data_path = os.path.join("..", "notebooks")
merged_data = os.path.join(merged_data_path, "merged_car_data.csv")
flattened_df = pd.read_csv(merged_data)


# Parse 'new_car_detail' column to extract price, km etc..
def safe_eval(val):
    if isinstance(val, str):
        try:
            return eval(val)
        except:
            return
    elif isinstance(val, dict):
        return val
    else:
        return

detail_dicts = flattened_df['new_car_detail'].apply(safe_eval)
detail_df = pd.DataFrame(detail_dicts.tolist(), columns= ['price', 'km'])

# clean 'price': remove currency symbols, 'Lakh', and convert to float
def clean_price(value):
    if isinstance(value, str):
        value = value.replace("₹", "").replace(",", "").replace("Lakh", "").strip()
        try:
            return float(value) * 100000 # Convert Lakh to actual amount
        except:
            return None
    return None

# clean 'km': remove 'kms' or commas and convert to int
def _clean_km(value):
    if isinstance(value, str):
        value = value.replace(",", "").replace("kms", "").replace("km", "").strip()
        try:
            return int(value)
        except:
            return None
    return None

# Apply cleaning functions
detail_df['price_cleaned'] = detail_df['price'].apply(clean_price)
detail_df['km_cleaned'] = detail_df['km'].apply(_clean_km)

# Print the DataFrame after cleaning functions are applied
print(detail_df[['price', 'price_cleaned', 'km', 'km_cleaned']].head(10).to_string(index=False))


      price  price_cleaned       km  km_cleaned
   ₹ 4 Lakh       400000.0 1,20,000      120000
₹ 8.11 Lakh       811000.0   32,706       32706
₹ 5.85 Lakh       585000.0   11,949       11949
₹ 4.62 Lakh       462000.0   17,794       17794
₹ 7.90 Lakh       790000.0   60,000       60000
  ₹ 19 Lakh      1900000.0   20,000       20000
₹ 3.45 Lakh       345000.0   37,772       37772
  ₹ 12 Lakh      1200000.0   30,000       30000
₹ 9.60 Lakh       960000.0   37,000       37000
₹ 5.85 Lakh       585000.0   11,949       11949


In [29]:
full_detail_df = pd.DataFrame(detail_dicts.tolist())
# Add city and car_links
full_detail_df['City'] = merged_df['City']
full_detail_df['car_links'] = merged_df['car_links']

# Check column info and missing values
column_info = full_detail_df.describe(include='all').transpose()
missing_values = full_detail_df.isnull().sum()
print(missing_values)

it                     0
ft                     0
bt                     0
km                     0
transmission           0
ownerNo                0
owner                  0
oem                    0
model                  0
modelYear              0
centralVariantId       0
variantName            0
price                  0
priceActual            0
priceSaving            0
priceFixedText      8369
trendingText           0
City                   0
car_links              0
dtype: int64


In [30]:
print(full_detail_df)

      it      ft         bt        km transmission  ownerNo      owner  \
0      0  Petrol  Hatchback  1,20,000       Manual        3  3rd Owner   
1      0  Petrol        SUV    32,706       Manual        2  2nd Owner   
2      0  Petrol  Hatchback    11,949       Manual        1  1st Owner   
3      0  Petrol      Sedan    17,794       Manual        1  1st Owner   
4      0  Diesel        SUV    60,000       Manual        1  1st Owner   
...   ..     ...        ...       ...          ...      ...        ...   
8364   0  Petrol  Hatchback    10,000       Manual        1  1st Owner   
8365   0  Petrol  Hatchback  1,20,000       Manual        1  1st Owner   
8366   0  Petrol      Sedan    50,000    Automatic        3  3rd Owner   
8367   0  Petrol  Hatchback    40,000       Manual        1  1st Owner   
8368   0  Diesel        SUV  1,20,000       Manual        2  2nd Owner   

                oem                  model  modelYear  centralVariantId  \
0            Maruti         Maruti C

In [32]:
# Select and rename relevant categorical columns for clarity
categorical_df = full_detail_df[['transmission', 'ft', 'owner']].rename(columns={
    'ft': 'fuel_type',
    'owner': 'ownership_status'
})

# Check unique values before encoding
transmission_unique = categorical_df['transmission'].unique().tolist()
fuel_type_unique = categorical_df['fuel_type'].unique().tolist()
ownership_unique = categorical_df['ownership_status'].unique().tolist()

# Apply label encoding for visualization
categorical_encoded = categorical_df.apply(lambda col: pd.factorize(col)[0])

# Combine with original for comparison
encoded_summary = pd.concat([categorical_df, categorical_encoded.add_suffix("_encoded")],axis=1)

print(encoded_summary)

     transmission fuel_type ownership_status  transmission_encoded  \
0          Manual    Petrol        3rd Owner                     0   
1          Manual    Petrol        2nd Owner                     0   
2          Manual    Petrol        1st Owner                     0   
3          Manual    Petrol        1st Owner                     0   
4          Manual    Diesel        1st Owner                     0   
...           ...       ...              ...                   ...   
8364       Manual    Petrol        1st Owner                     0   
8365       Manual    Petrol        1st Owner                     0   
8366    Automatic    Petrol        3rd Owner                     1   
8367       Manual    Petrol        1st Owner                     0   
8368       Manual    Diesel        2nd Owner                     0   

      fuel_type_encoded  ownership_status_encoded  
0                     0                         0  
1                     0                         1  
2  

### 📉**Descriptive Statistics**

In [37]:
# Compute car age assuming current year is 2025
# Apply cleaning and create derived fields
full_detail_df['price_cleaned'] = full_detail_df['price'].apply(clean_price)
full_detail_df['km_cleaned'] = full_detail_df['km'].apply(_clean_km)
full_detail_df['car_age'] = 2025 - full_detail_df['modelYear']

# Compute descriptive statistics on numeric columns
numeric_cols = full_detail_df[['price_cleaned', 'km_cleaned', 'car_age', 'ownerNo']]
numeric_stats = numeric_cols.describe().transpose()

print(f"📉 Descriptive Statistics of Numerical Columns:")
print(numeric_stats.to_string())

📉 Descriptive Statistics of Numerical Columns:
                count          mean           std       min       25%       50%       75%           max
price_cleaned  8349.0  8.911557e+07  7.924329e+08  100000.0  403000.0  620000.0  978000.0  9.970300e+09
km_cleaned     8369.0  5.897430e+04  7.406100e+04       0.0   30000.0   53692.0   80000.0  5.500000e+06
car_age        8369.0  8.496714e+00  3.921465e+00       2.0       6.0       8.0      11.0  4.000000e+01
ownerNo        8369.0  1.360139e+00  6.419585e-01       0.0       1.0       1.0       2.0  5.000000e+00


### 📊 **Numerical Summary**

| Feature            | Mean      | Std Dev   | Min   | 25%    | Median | 75%    | Max       |
|--------------------|-----------|-----------|--------|--------|--------|--------|------------|
| **price_cleaned** (₹) | ₹89.1 L | ₹792.4 L | ₹1 L   | ₹4.03 L | ₹6.2 L  | ₹9.78 L | ₹99.7 Cr ⚠️ |
| **km_cleaned**        | 58,974   | 74,061    | 0     | 30,000 | 53,692 | 80,000 | 5.5 L km ⚠️ |
| **car_age** (yrs)     | 8.5      | 3.92      | 2     | 6      | 8      | 11     | 40 yrs ⚠️ |
| **ownerNo**           | 1.36     | 0.64      | 0     | 1      | 1      | 2      | 5 owners   |

**⚠️ Observations:**   
- Price and km have outliers — worth visualizing with box plots.
- Car Age max is 40 — very rare; possibly needs validation.
- OwnerNo suggests most cars are 1st or 2nd hand.